In [6]:
from queue import Queue, PriorityQueue
import time

In [7]:
def read_input(filename):
    with open(filename, 'r') as f:
        d = {}
        for (i, l) in enumerate(f.readlines()):
            for (j, ll) in enumerate(l[:-1]):
                d[(i, j)] = ll
    return d

def get_key(d, val):
    for (key, value) in d.items():
        if value == val:
            return key
    return 'Start not found'

In [26]:
def get_adjacent(p, tunnel_map, all_keys):
    (x, y), keys = p
    doors = [k.upper() for k in keys]
    adj = []
    for p_a in [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]:
        if tunnel_map[p_a] in ['.', '@'] + doors + all_keys:
            adj.append((p_a, p[1]))
    return adj

def is_explored(explored, w):
    return any([w[0] == e[0] and set(w[1]) == set(e[1]) for e in explored])

def bfs(tunnel_map, p_start, all_keys):
    q = Queue()
    q.put(p_start)
    explored = set()
    dist = {p_start: 0}
    while not q.empty():
        v = q.get()
        xy, keys = v
        if tunnel_map[xy] in all_keys:
            if not tunnel_map[xy] in keys:
                v_old = v
                v = (xy, keys | frozenset(tunnel_map[xy]))
                dist[v] = dist[v_old]
        if set(v[1]) == set(all_keys):
            return dist[v]
        for w in get_adjacent(v, tunnel_map, all_keys):
            if not w in explored:
                explored.add(w)
                dist[w] = dist[v] + 1
                q.put(w)
    return -1

def dijkstra(tunnel_map, p_start, all_keys):
    dist = {p_start: 0}
    prev = {}
    q = PriorityQueue()
    q.put((dist[p_start], p_start))
    while not q.empty():
        d, v = q.get()
        xy, keys = v
        if tunnel_map[xy] in all_keys:
            if not tunnel_map[xy] in keys:
                v_old = v
                v = (xy, keys | frozenset(tunnel_map[xy]))
                dist[v] = dist[v_old]
        if set(v[1]) == set(all_keys):
            return d
        for w in get_adjacent(v, tunnel_map, all_keys):
            alt = dist[v] + 1
            if alt < dist.get(w, 1000000):
                dist[w] = alt
                prev[w] = v
                if w not in [val[1] for val in q.queue]:
                    q.put((alt, w))
    return -1


In [27]:
def runit(filename, alg = bfs):
    tunnel_map = read_input(filename)
    all_keys = [c for c in list(set(tunnel_map.values()) - set(['#', '.', '@'])) if c.islower()]
    p_start = get_key(tunnel_map, '@')
    t0 = time.time()
    dist = alg(tunnel_map, (p_start, frozenset()), all_keys)
    dt = time.time() - t0
    res = {'time': dt, 'distance': dist}
    return res

In [32]:
res = runit('18_input.txt')
res

{'time': 8.283371448516846, 'distance': 4350}